In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [4]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [5]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [6]:
outputs

[tensor([[[[0.2826, 0.9714, 1.5220,  ..., 0.7373, 0.9289, 0.4099],
           [0.4624, 0.9140, 1.0420,  ..., 0.0514, 0.6179, 0.4873],
           [0.0000, 0.8100, 0.2740,  ..., 1.3211, 1.1572, 0.5283],
           ...,
           [0.3495, 0.1663, 0.6029,  ..., 0.5366, 0.6767, 0.7779],
           [0.6466, 0.2827, 0.2755,  ..., 0.6202, 0.0000, 0.1652],
           [0.5089, 0.0647, 0.0000,  ..., 1.1579, 0.0000, 0.8187]],
 
          [[0.4079, 0.9405, 1.2423,  ..., 1.2189, 0.0000, 0.1604],
           [0.0000, 0.4853, 0.1722,  ..., 0.2318, 0.3710, 0.6873],
           [0.7707, 0.0000, 0.3716,  ..., 0.2827, 0.8774, 1.3938],
           ...,
           [1.2235, 0.6412, 0.0000,  ..., 1.4741, 0.4246, 0.3686],
           [0.4636, 0.4482, 0.7489,  ..., 0.4843, 0.7810, 0.0000],
           [0.5311, 0.0709, 0.6119,  ..., 0.3238, 0.7142, 0.8323]],
 
          [[0.2491, 0.3374, 0.9442,  ..., 0.9518, 1.3392, 1.1033],
           [1.3463, 0.9497, 1.1240,  ..., 0.4639, 1.6020, 1.1765],
           [0.6703, 0.97

In [7]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [14]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
          torch.nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [15]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


